In [1]:
with open('BabyFlightNodes.txt', mode='r') as BabyFlightNodes:
    FlightEdges = BabyFlightNodes.read().split("%SECTION%")[2]
with open('BabyFlightNodes.txt', mode='r') as BabyFlightNodes:
    FlightNodes = BabyFlightNodes.read().split("%SECTION%")[1]

In [2]:
with open('BabyFlight.txt', mode='r') as BabyFlight:
    Flight1 = BabyFlight.read().split("\n")[1]

In [3]:
from ipynb.fs.full.GetTrajectory import GetTrajectory
from ipynb.fs.full.PerpendicularDistance import NearestPointOnLine
from ipynb.fs.full.PerpendicularDistance import GetPerpendicularDistance

In [4]:
FlightTrajectoryNodes = GetTrajectory(Flight1)

In [5]:
from ipynb.fs.full.GetTaxiway import GetTaxiway
from ipynb.fs.full.GetTaxiway import GetEdgeID
from ipynb.fs.full.GetTaxiway import GetEdgeStart
from ipynb.fs.full.GetTaxiway import GetEdgeEnd
from ipynb.fs.full.GetTaxiway import GetEdgeLength
from ipynb.fs.full.GetTaxiway import GetEdgeSpecification

In [6]:
FlightNodesList = GetTaxiway(FlightNodes, FlightEdges)[0]
FlightEdgesList = GetTaxiway(FlightNodes, FlightEdges)[1]
FlightEdgesWeights = GetTaxiway(FlightNodes, FlightEdges)[2]

In [7]:
GetEdgeEnd(FlightEdgesList, FlightNodesList)

[('12.0', '23.0'),
 ('17.0', '19.0'),
 ('24.0', '18.0'),
 ('27.0', '18.0'),
 ('38.0', '18.0'),
 ('42.0', '19.0'),
 ('44.0', '22.0'),
 ('45.0', '29.0'),
 ('28.0', '23.0'),
 ('33.0', '27.0'),
 ('35.0', '28.0'),
 ('39.0', '30.0'),
 ('45.0', '29.0'),
 ('50.0', '29.0')]

In [8]:

for item in FlightTrajectoryNodes:
    print(item)
    for index, edge in enumerate(FlightEdgesList):
        EdgeID = GetEdgeID(FlightEdgesList)[index]
        EdgeStart = GetEdgeStart(FlightEdgesList, FlightNodesList)[index]
        EdgeEnd = GetEdgeEnd(FlightEdgesList, FlightNodesList)[index]
        EdgeLength = GetEdgeLength(FlightEdgesList)[index]
            
        PointX = float(item[0])
        PointY = float(item[1])
        StartX = float(EdgeStart[0])
        StartY = float(EdgeStart[1])
        EndX = float(EdgeEnd[0])
        EndY = float(EdgeEnd[1])
            
        (IntX, IntY) = NearestPointOnLine(PointX, PointY, StartX, StartY, EndX, EndY)
        
        PDistance = GetPerpendicularDistance(PointX, PointY, (IntX, IntY)[0], (IntX, IntY)[1])
        
        print(index, EdgeID, StartX, StartY, EndX, EndY, (IntX,IntY), PDistance)
            
        pass

('12.0', '29.0')
0 1 8.0 30.0 12.0 23.0 (9.415384615384617, 27.523076923076925) 2.9768336301410003
1 2 12.0 23.0 17.0 19.0 (9.073170731707318, 25.34146341463415) 4.68521285665818
2 3 17.0 19.0 24.0 18.0 (10.7, 19.89999999999999) 9.192388155425126
3 4 24.0 18.0 27.0 18.0 (12.0, 18.0) 11.0
4 5 27.0 18.0 38.0 18.0 (12.0, 18.0) 11.0
5 6 38.0 18.0 42.0 19.0 (16.11764705882353, 12.529411764705884) 16.977493752543307
6 7 42.0 19.0 44.0 22.0 (37.38461538461539, 12.076923076923077) 30.508510792387604
7 8 44.0 22.0 45.0 29.0 (44.339999999999996, 24.380000000000003) 32.668333290818495
8 9 27.0 18.0 28.0 23.0 (28.53846153846154, 25.69230769230769) 16.86598762188383
9 10 28.0 23.0 33.0 27.0 (21.170731707317078, 17.536585365853654) 14.680333617528976
10 11 33.0 27.0 35.0 28.0 (17.0, 19.0) 11.180339887498949
11 12 35.0 28.0 39.0 30.0 (17.0, 19.0) 11.180339887498949
12 13 39.0 30.0 45.0 29.0 (12.891891891891891, 34.351351351351354) 5.425166581076794
13 14 45.0 29.0 50.0 29.0 (12.0, 29.0) 0.0
('16.0', 